In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import matplotlib.pyplot as plt
import statsmodels.tsa.seasonal as smt
import statsmodels.tsa.arima_model as arm
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import datetime as dt
from sklearn import linear_model 
import plotly.plotly as py
import plotly.tools as tls

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))
import os
os.chdir('../input/Data/Stocks/')

In [ ]:
#read data
# kernels let us navigate through the zipfile as if it were a directory

# trying to read a file of size zero will throw an error, so skip them
filenames = [x for x in os.listdir() if x.endswith('.txt') and os.path.getsize(x) > 0]
#print(filenames)
#filenames = random.sample(filenames,10)
filenames = ['prk.us.txt', 'bgr.us.txt', 'fr.us.txt', 'star.us.txt', 'sons.us.txt', 'utg.us.txt']
print(filenames)

In [ ]:
data = []
for filename in filenames:
    df = pd.read_csv(filename, sep=',')
    #combine attributes
    #combine high and low by avg
    #combine open and close by avg
    #combine avgHighLow and avgOpenClose
    df['Price'] = (df['High'] + df['Low'] + df['Open'] + df['Close'])/4
    #take log as this flattens the data more, resulting in a better prediction
    df['Price'] = np.log(df['Price'])

    #drop obsolete columns for faster processing
    columns2Drop = ['High', 'Low', 'Open', 'Close', 'OpenInt']
    df = df.drop(labels=columns2Drop, axis=1)

    #create new attribute of "movement"
    df['Volume*Price'] = df['Volume'] * df['Price']
    # print(df)

    label, _, _ = filename.split(sep='.')
    df['Label'] = filename
    df['Date'] = pd.to_datetime(df['Date'])
    #conver data to an number so we can fit it to LinearRegression()
#     df['Date'] = df['Date'].map(dt.datetime.toordinal)

    data.append(df)
# print(data)

Attempt at making data stationary by taking residuals from a fitted model. Does not work well?

In [ ]:
def applyAttributes(X,attributeMod):
    for func in attributeMod:
        modif = [func(x) for x in X[:,0]]
        X = np.c_[X, modif]
    return X 


def normalize(X):
    return np.array((X - np.mean(X))/np.std(X))

for df in data:
    X = df['Date'].map(dt.datetime.toordinal).values.reshape(-1, 1)
    y = df['Price'].values.reshape(-1, 1)
    Xnorm = normalize(X)
    ynorm = normalize(y)
 
    attributeMod = []
    #attributeMod = [lambda x:  x**2, lambda x: np.sign(x) * x**2, lambda x: x**3,
                   # lambda x: np.sqrt(np.abs(x)) * np.sign(x), lambda x: np.cbrt(x),
                    #lambda x:np.sin(x),lambda x:np.cos(x),lambda x:np.sin(5*x),lambda x:np.cos(5*x),
                   #lambda x: 2 ** x, lambda x: 3 ** x, lambda x: np.abs(x) ** x] 
                    #These are mostly tests
    
    Xmod = applyAttributes(Xnorm,attributeMod) 
    reg = linear_model.Ridge(alpha = 3)
    reg.fit(X=Xmod,y=ynorm)
    
    plt.plot(Xnorm, ynorm,color='c')
    plt.plot(Xnorm, reg.predict(applyAttributes(Xnorm,attributeMod)),color='r')
    plt.xlabel('Time')
    plt.ylabel('Price')
    plt.show()
    
    plt.plot(Xnorm, ynorm-reg.predict(applyAttributes(Xnorm,attributeMod)),color='k')
    plt.plot(Xnorm, ynorm,color='c')
    plt.xlabel('Time')
    plt.ylabel('Residual of price(Black) and Price(Cyan)')
    plt.show()

Attempt at making data stationary by differencing.

In [ ]:
import copy

def normalize(X):
    return np.array((X - np.mean(X))/np.std(X))

for df in data:
    X = df['Date'].map(dt.datetime.toordinal).values.reshape(-1, 1)
    y = df['Price'].values.reshape(-1, 1)
    Xnorm = normalize(X)
    ynorm = normalize(y)
 
    ydiffer = [0]
    for i in range(1,len(ynorm)):
        ydiffer.append(ynorm[i]-ynorm[i-1])
    

    
    plt.plot(Xnorm, ydiffer,color='k')
    plt.plot(Xnorm, ynorm,color='c')
    plt.xlabel('Time')
    plt.ylabel('Difference in price(Black) and Price(Cyan)')
    plt.show()
    
        #plt.scatter(Xnorm, ynorm,color='g')
    plt.plot(Xnorm, ydiffer,color='k')
    plt.xlabel('Time')
    plt.ylabel('Difference in price')
    plt.show()
    
    df['PriceNorm'] = ynorm
    df['DateNorm'] = Xnorm
    df['PriceStation'] = ydiffer
    

TEST ARMA:

In [ ]:
overallerror = np.zeros((8,8))
errorlist = []
for df in data:
    break;
    X = df['DateNorm'].reshape(-1, 1)
    y = df['PriceStation'].reshape(-1, 1)
    train = int(len(X)-5)
    Xtrain = X[:train]
    ytrain = y[:train]
    Xtest = X[train:]
    ytest = y[train:]
    
    errors = []
    for p in range(0,8):
        errorsthisp = []
        for q in range(0,8):
            model = arm.ARMA(ytrain,order=(p,0,q))
            model_fit = model.fit(disp=False,transparams=False)
            error = np.sum(np.abs(ytest-model_fit.forecast(len(Xtest))[0]))/8
            
            #print(error)
           # print(model_fit.forecast(len(Xtest))[0][0:20])
            if error > 100000:
                error = errorlist[-1]
            errorlist.append(error)   
            errorsthisp.append(error)
        errors.append(errorsthisp)
    overallerror = overallerror + np.array(errors)
    #print(np.array(errors).shape) 
    
    #plt.imshow(np.array(errors), cmap='hot', interpolation='nearest')
    #plt.show()
    #ypred
    #print(model_fit.summary())
    # plot residual errors
    #print(type(model_fit.forecast()[0]))
    #print(model_fit.forecast())
    #residuals = pd.DataFrame(model_fit.resid)
    #residuals.plot()
    #plt.show()
    #residuals.plot(kind='kde')
    #plt.show()
    #print(residuals.describe())
    #print()
    #plt.plot(X, y,color='c')
    #plt.plot(Xtrain, model_fit.fittedvalues,color='k')
    #plt.plot(Xtest, model_fit.forecast(len(Xtest))[0],color='r')
    #plt.xlabel('Time')
    #plt.ylabel('Difference in price')
    #plt.show()
    
    
    #print('score', reg.score(applyAttributes(Xtest,attributeMod),ytest))
#overallerror = np.array(overallerror)
print(errorlist)
plt.xlabel('q')
plt.ylabel('p')
plt
plt.imshow(overallerror/len(data), cmap='hot', interpolation='nearest')
plt.colorbar()
plt.show()

Try linear regression.

In [ ]:
for df in data:
    X = df['DateNorm'].reshape(-1, 1)
    y = df['PriceNorm'].reshape(-1, 1)
    train = int(len(X)-5)
    Xtrain = X[:train]
    ytrain = y[:train]
    Xtest = X[train:]
    ytest = y[train:]
    
    reg = linear_model.LinearRegression()
    reg.fit(X=Xtrain,y=ytrain)

    plt.plot(X, y,color='c')
    plt.plot(Xtrain, reg.predict(Xtrain),color='k')
    plt.plot(Xtest, reg.predict(Xtest),color='r')
    plt.xlabel('Time')
    plt.ylabel('Differene in price')
    
    print('score', reg.score(Xtest,ytest))

    plt.show()

In [ ]:
def applyAttributes(X,attributeMod):
    for func in attributeMod:
        modif = [func(x) for x in X[:,0]]
        X = np.c_[X, modif]
    return X 

for df in data:
    X = df['DateNorm'].reshape(-1, 1)
    y = df['PriceNorm'].reshape(-1, 1)
    train = int(len(X)-5)
    Xtrain = X[:train]
    ytrain = y[:train]
    Xtest = X[train:]
    ytest = y[train:]
    
    attributeMod = [lambda x:  x**2, lambda x: x**3,
                    lambda x: np.sqrt(np.abs(x)) * np.sign(x), lambda x: np.cbrt(x) * np.sign(x),
                    lambda x:np.sin(x),lambda x:np.cos(x),lambda x:np.sin(5*x),lambda x:np.cos(5*x),
                    lambda x:np.sin(0.2*x),lambda x:np.cos(0.2*x),
                   lambda x: 2 ** x, lambda x: 3 ** x, lambda x: np.abs(x) ** x] 
                    #These are mostly tests
    
    Xmodtrain = applyAttributes(Xtrain,attributeMod) 
    reg = linear_model.LinearRegression()
    reg.fit(X=Xmodtrain,y=ytrain)
    
    plt.plot(X, y,color='c')
    plt.plot(X, reg.predict(applyAttributes(X,attributeMod)),color='k')
    plt.plot(Xtest, reg.predict(applyAttributes(Xtest,attributeMod)),color='r')
    plt.title('Fitted multiple regression model')
    plt.xlabel('Time')
    plt.ylabel('Price')
    plt.show()
    
    print('score', reg.score(applyAttributes(Xtest,attributeMod),ytest))

    

As the multiple regression model is (horribly) overfit, we use a ridge regression model for regularization. We use a special version of CV specialized for time series.

In [ ]:
def applyAttributes(X,attributeMod):
    for func in attributeMod:
        modif = [func(x) for x in X[:,0]]
        X = np.c_[X, modif]
    return X 

def crossValidate(X,y,alpha):
    sumScores = 0
    for vold in range(1,10):
        train = int(vold/10*len(X))
        test = int((vold+1)/10*len(X))
        Xtrain = X[:train] 
        ytrain = y[:train] 
        Xtest = X[train:test] 
        ytest = y[train:test] 
        
        reg = linear_model.Ridge(alpha = alpha)
        reg.fit(X=Xtrain,y=ytrain)
        sumScores += np.abs(reg.score(Xtest,ytest))
    return sumScores/9
        
MAE = []        
for f in range(1,11):
    thisf = []
    for df in data:
        X = df['DateNorm'].reshape(-1, 1)
        y = df['PriceNorm'].reshape(-1, 1)
        train = int(len(X)-f)
        Xtrain = X[:train]
        ytrain = y[:train]
        Xtest = X[train:]
        ytest = y[train:]
    
        attributeMod = [lambda x:  x**2, lambda x: np.sign(x) * x**2, lambda x: x**3,
                    lambda x: np.sqrt(np.abs(x)) * np.sign(x), lambda x: np.cbrt(x),
                    lambda x:np.sin(x),lambda x:np.cos(x),lambda x:np.sin(5*x),lambda x:np.cos(5*x),
                   lambda x: 2 ** x, lambda x: 3 ** x, lambda x: np.abs(x) ** x] 
                    #These are mostly tests
    
        Xmodtrain = applyAttributes(Xtrain,attributeMod) 
    
    #averages = []
    #for alpha in np.arange(0.5, 30.5, 0.5):
    #    averages.append(crossValidate(Xmodtrain,ytrain,alpha))
    #plt.plot(np.arange(0.5, 30.5, 0.5), averages)
    #plt.xlabel('Alpha')
    #plt.ylabel('Error')
    #plt.title('Test error against alpha')
    #plt.show()

        reg = linear_model.Ridge(alpha = 25.0)
        reg.fit(X=Xmodtrain,y=ytrain)
        thisf.append(np.mean(np.abs(reg.predict(applyAttributes(Xtest,attributeMod) )-ytest)))
        #plt.plot(X, y,color='c')
        #plt.plot(X, reg.predict(applyAttributes(X,attributeMod)),color='k')
        #plt.plot(Xtest, reg.predict(applyAttributes(Xtest,attributeMod)),color='r')
        #plt.title('Fitted multiple regression model')
        #plt.xlabel('Time')
        #plt.ylabel('Difference in price')
        #plt.show()
    MAE.append(np.mean(thisf))
plt.plot(np.arange(1, 11, 1), MAE)
plt.xlabel('Amount of forecasting points')
plt.ylabel('MAE')
plt.title('MAE against the amount of forecasting points')
plt.show()  
    #print('score', reg.score(applyAttributes(Xtest,attributeMod),ytest))

Try exponential smoothing. FAILED: It is not a forecasting algorithm

In [ ]:
'''def crossValidate(X,y,alpha):
    sumScores = 0
    for vold in range(1,10):
        train = int(vold/10*len(X))
        test = int((vold+1)/10*len(X))
        Xtrain = X[:train] 
        ytrain = y[:train] 
        Xtest = X[train:test] 
        ytest = y[train:test] 
        
        span = int(2/alpha -1) #alpha = 2/(span+1)
        pred =  pd.ewma(np.append(ytrain,np.zeros(len(ytest))),span=span)
        print(vold)
        sumScores += np.sum((pred[len(ytest):]-ytest)**2)
    return sumScores/9
        
        

for df in data:
    X = df['DateNorm'].reshape(-1, 1)
    y = df['PriceStation'].reshape(-1, 1)
    train = int(0.95*len(X))
    Xtrain = X[:train]
    ytrain = y[:train]
    Xtest = X[train:]
    ytest = y[train:]
    

    averages = []
    for alpha in np.arange(0.10, 1.00, 0.10):
        averages.append(crossValidate(Xtrain,ytrain,alpha))
    plt.plot(np.arange(0.10, 1.00, 0.10), averages)
    plt.xlabel('Sum of Square Errors')
    plt.ylabel('Error')
    plt.show()
    
    alpha = 0.2
    span = int(2/alpha -1) #alpha = 2/(span+1)
    pred =  pd.ewma(np.append(ytrain,np.zeros(len(ytest))),span=span)
    
    print(len(pred))
    print(len(ytrain))
    plt.plot(X, y,color='g')
    plt.plot(X, pred,color='k')
    plt.plot(Xtest, pred[len(ytest):],color='r')
    plt.xlabel('Time')
    plt.ylabel('Difference in price')
    plt.show()
    
    '''
    #print('score', reg.score(applyAttributes(Xtest,attributeMod),ytest))

ARMA: (Not ARIMA, as we already differentiated the data)

In [ ]:
MAE = []        
for f in range(1,11):
    thisf = []
    for df in data:
        X = df['DateNorm'].reshape(-1, 1)
        y = df['PriceStation'].reshape(-1, 1)
        train = int(len(X)-f)
        Xtrain = X[:train]
        ytrain = y[:train]
        Xtest = X[train:]
        ytest = y[train:]
    
    #averages = []
    #for alpha in np.arange(0.5, 30.5, 0.5):
    #    averages.append(crossValidate(Xmodtrain,ytrain,alpha))
    #plt.plot(np.arange(0.5, 30.5, 0.5), averages)
    #plt.xlabel('Alpha')
    #plt.ylabel('Error')
    #plt.title('Test error against alpha')
    #plt.show()

        model = arm.ARMA(ytrain,order=(3,0,3))
        model_fit = model.fit(disp=False,transparams=False)
        error = np.mean(np.abs(model_fit.forecast(len(Xtest))[0]-ytest))
        if error > 10000:
            error = thisf[-1]
        thisf.append(error)
        #plt.plot(X, y,color='c')
        #plt.plot(X, reg.predict(applyAttributes(X,attributeMod)),color='k')
        #plt.plot(Xtest, reg.predict(applyAttributes(Xtest,attributeMod)),color='r')
        #plt.title('Fitted multiple regression model')
        #plt.xlabel('Time')
        #plt.ylabel('Difference in price')
        #plt.show()
    MAE.append(np.mean(thisf))
plt.plot(np.arange(1, 11, 1), MAE)
plt.xlabel('Amount of forecasting points')
plt.ylabel('MAE')
plt.title('MAE against the amount of forecasting points')
plt.show()  